# MoA

Fork from [here](https://www.kaggle.com/simakov/keras-multilabel-neural-network-v1-2)

In [1]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import os
import random

import numpy as np
import pandas as pd

import optuna
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow_addons as tfa
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.utils import check_random_state
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tqdm.notebook import tqdm

## 関数

In [3]:
def fix_seed(seed=2020):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


seed = 2020
fix_seed(seed)

In [4]:
def preprocess(df):
    df = df.copy()
    df.loc[:, "cp_type"] = df.loc[:, "cp_type"].map({"trt_cp": 0, "ctl_vehicle": 1})
    df.loc[:, "cp_dose"] = df.loc[:, "cp_dose"].map({"D1": 0, "D2": 1})
    del df["sig_id"]
    return df

In [5]:
def iter_shuffled(X, columns_to_shuffle=None, pre_shuffle=False, random_state=None):
    rng = check_random_state(random_state)

    if columns_to_shuffle is None:
        columns_to_shuffle = range(X.shape[1])

    if pre_shuffle:
        X_shuffled = X.copy()
        rng.shuffle(X_shuffled)

    X_res = X.copy()
    for columns in tqdm(columns_to_shuffle):
        if pre_shuffle:
            X_res[:, columns] = X_shuffled[:, columns]
        else:
            rng.shuffle(X_res[:, columns])
        yield X_res
        X_res[:, columns] = X[:, columns]

In [6]:
def get_score_importances(
    score_func,  # type: Callable[[Any, Any], float]
    X,
    y,
    n_iter=5,  # type: int
    columns_to_shuffle=None,
    random_state=None,
):
    rng = check_random_state(random_state)
    base_score = score_func(X, y)
    scores_decreases = []
    for i in range(n_iter):
        scores_shuffled = _get_scores_shufled(
            score_func, X, y, columns_to_shuffle=columns_to_shuffle, random_state=rng, base_score=base_score
        )
        scores_decreases.append(scores_shuffled)

    return base_score, scores_decreases

In [7]:
def _get_scores_shufled(score_func, X, y, base_score, columns_to_shuffle=None, random_state=None):
    Xs = iter_shuffled(X, columns_to_shuffle, random_state=random_state)
    res = []
    for X_shuffled in Xs:
        res.append(-score_func(X_shuffled, y) + base_score)
    return res

In [8]:
"""
def metric(y_true, y_pred):
    metrics = []
    for i in range(y_pred.shape[1]):
        if y_true[:, i].sum() > 1:
            metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float)))
    return np.mean(metrics)
"""

def metric(y_true, y_pred):
    metrics = []
    for _target in target_df.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)


## データロード

In [9]:
train_df = pd.read_csv("../input/lish-moa/train_features.csv")
test_df = pd.read_csv("../input/lish-moa/test_features.csv")
target_df = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
non_target_df = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
submit_df = pd.read_csv("../input/lish-moa/sample_submission.csv")

## 前処理

In [10]:
train = preprocess(train_df)
test = preprocess(test_df)

In [11]:
del target_df["sig_id"]

### cp_type が ctrl_vehicle なものは MoA を持たない

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 875 entries, cp_type to c-99
dtypes: float64(872), int64(3)
memory usage: 159.0 MB


In [13]:
train[train["cp_type"] == 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21948 entries, 0 to 23813
Columns: 875 entries, cp_type to c-99
dtypes: float64(872), int64(3)
memory usage: 146.7 MB


In [14]:
target_df = target_df.loc[train["cp_type"] == 0].reset_index(drop=True)

In [15]:
train = train.loc[train["cp_type"] == 0].reset_index(drop=True)

## モデル レイヤ定義

In [16]:
def create_layer(units, dropout_rate):
    layer = [
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate),
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(units, activation="relu")),        
    ]
    return layer

## Features Importance ??

In [17]:
"""
perm_imp = np.zeros(train.shape[1])
all_res = []
for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=0, shuffle=True).split(target_df)):
    print(f"Fold {n}")

    model = create_model(len(train.columns))
    checkpoint_path = f"repeat:{seed}_Fold:{n}.hdf5"
    reduce_lr_loss = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode="min")
    cb_checkpt = ModelCheckpoint(
        checkpoint_path, monitor="val_loss", verbose=0, save_best_only=True, save_weights_only=True, mode="min"
    )
    model.fit(
        train.values[tr],
        target_df.values[tr],
        validation_data=(train.values[te], target_df.values[te]),
        epochs=35,
        batch_size=128,
        callbacks=[reduce_lr_loss, cb_checkpt],
        verbose=2,
    )

    model.load_weights(checkpoint_path)

    def _score(X, y):
        pred = model.predict(X)
        return metric(y, pred)

    base_score, local_imp = get_score_importances(
        _score, train.values[te], target_df.values[te], n_iter=1, random_state=0
    )
    all_res.append(local_imp)
    perm_imp += np.mean(local_imp, axis=0)
    print("")

top_feats = np.argwhere(perm_imp < 0).flatten()
top_feats
"""

'\nperm_imp = np.zeros(train.shape[1])\nall_res = []\nfor n, (tr, te) in enumerate(KFold(n_splits=7, random_state=0, shuffle=True).split(target_df)):\n    print(f"Fold {n}")\n\n    model = create_model(len(train.columns))\n    checkpoint_path = f"repeat:{seed}_Fold:{n}.hdf5"\n    reduce_lr_loss = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode="min")\n    cb_checkpt = ModelCheckpoint(\n        checkpoint_path, monitor="val_loss", verbose=0, save_best_only=True, save_weights_only=True, mode="min"\n    )\n    model.fit(\n        train.values[tr],\n        target_df.values[tr],\n        validation_data=(train.values[te], target_df.values[te]),\n        epochs=35,\n        batch_size=128,\n        callbacks=[reduce_lr_loss, cb_checkpt],\n        verbose=2,\n    )\n\n    model.load_weights(checkpoint_path)\n\n    def _score(X, y):\n        pred = model.predict(X)\n        return metric(y, pred)\n\n    base_score, local_imp = get_score_importances(

### 3分クッキング

In [18]:
# https://www.kaggle.com/gogo827jz/split-neural-network-approach-tf-keras
top_feats = [  0,   1,   2,   3,   5,   6,   8,   9,  10,  11,  12,  14,  15,
        16,  18,  19,  20,  21,  23,  24,  25,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  39,  40,  41,  42,  44,  45,  46,
        48,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        63,  64,  65,  66,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        78,  79,  80,  81,  82,  83,  84,  86,  87,  88,  89,  90,  92,
        93,  94,  95,  96,  97,  99, 100, 101, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       149, 150, 151, 152, 153, 154, 155, 157, 159, 160, 161, 163, 164,
       165, 166, 167, 168, 169, 170, 172, 173, 175, 176, 177, 178, 180,
       181, 182, 183, 184, 186, 187, 188, 189, 190, 191, 192, 193, 195,
       197, 198, 199, 202, 203, 205, 206, 208, 209, 210, 211, 212, 213,
       214, 215, 218, 219, 220, 221, 222, 224, 225, 227, 228, 229, 230,
       231, 232, 233, 234, 236, 238, 239, 240, 241, 242, 243, 244, 245,
       246, 248, 249, 250, 251, 253, 254, 255, 256, 257, 258, 259, 260,
       261, 263, 265, 266, 268, 270, 271, 272, 273, 275, 276, 277, 279,
       282, 283, 286, 287, 288, 289, 290, 294, 295, 296, 297, 299, 300,
       301, 302, 303, 304, 305, 306, 308, 309, 310, 311, 312, 313, 315,
       316, 317, 320, 321, 322, 324, 325, 326, 327, 328, 329, 330, 331,
       332, 333, 334, 335, 338, 339, 340, 341, 343, 344, 345, 346, 347,
       349, 350, 351, 352, 353, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 368, 369, 370, 371, 372, 374, 375, 376, 377,
       378, 379, 380, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391,
       392, 393, 394, 395, 397, 398, 399, 400, 401, 403, 405, 406, 407,
       408, 410, 411, 412, 413, 414, 415, 417, 418, 419, 420, 421, 422,
       423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435,
       436, 437, 438, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450,
       452, 453, 454, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 468, 469, 471, 472, 473, 474, 475, 476, 477, 478, 479, 482,
       483, 485, 486, 487, 488, 489, 491, 492, 494, 495, 496, 500, 501,
       502, 503, 505, 506, 507, 509, 510, 511, 512, 513, 514, 516, 517,
       518, 519, 521, 523, 525, 526, 527, 528, 529, 530, 531, 532, 533,
       534, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547,
       549, 550, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563,
       564, 565, 566, 567, 569, 570, 571, 572, 573, 574, 575, 577, 580,
       581, 582, 583, 586, 587, 590, 591, 592, 593, 595, 596, 597, 598,
       599, 600, 601, 602, 603, 605, 607, 608, 609, 611, 612, 613, 614,
       615, 616, 617, 619, 622, 623, 625, 627, 630, 631, 632, 633, 634,
       635, 637, 638, 639, 642, 643, 644, 645, 646, 647, 649, 650, 651,
       652, 654, 655, 658, 659, 660, 661, 662, 663, 664, 666, 667, 668,
       669, 670, 672, 674, 675, 676, 677, 678, 680, 681, 682, 684, 685,
       686, 687, 688, 689, 691, 692, 694, 695, 696, 697, 699, 700, 701,
       702, 703, 704, 705, 707, 708, 709, 711, 712, 713, 714, 715, 716,
       717, 723, 725, 727, 728, 729, 730, 731, 732, 734, 736, 737, 738,
       739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751,
       752, 753, 754, 755, 756, 758, 759, 760, 761, 762, 763, 764, 765,
       766, 767, 769, 770, 771, 772, 774, 775, 780, 781, 782, 783, 784,
       785, 787, 788, 790, 793, 795, 797, 799, 800, 801, 805, 808, 809,
       811, 812, 813, 816, 819, 820, 821, 822, 823, 825, 826, 827, 829,
       831, 832, 833, 834, 835, 837, 838, 839, 840, 841, 842, 844, 845,
       846, 847, 848, 850, 851, 852, 854, 855, 856, 858, 860, 861, 862,
       864, 867, 868, 870, 871, 873, 874]
print(len(top_feats))

696


## 学習(ハイパーパラメータチューニング)

In [19]:
class Objective:
    def __init__(self):
        self.best_submit_df = None
        self._submit_df = None
    
    def __call__(self, trial):
        # Clear clutter from previous Keras session graphs.
        K.clear_session()
        
        n_starts = trial.suggest_int("n_starts", 1, 10)
        n_kfold = trial.suggest_int("n_kfold", 3, 10)
        n_layers = trial.suggest_int("n_layers", 1, 8)
        
        layers = []
        for layer in range(n_layers):
            unit = trial.suggest_categorical(f"units_{layer+1}", [2048, 1024, 512, 256])
            #drop_rate = trial.suggest_loguniform(f"drop_rate_{layer+1}", 0.2, 0.5)
            drop_rate = 0.5
            layers.extend(create_layer(unit, drop_rate))
        
        print(f"n_starts: {n_starts}, n_kfold: {n_kfold}, n_layers: {n_layers}")
        
        res = target_df.copy()
        submit_df.loc[:, target_df.columns] = 0
        res.loc[:, target_df.columns] = 0

        for seed in range(n_starts):
            for n, (tr, te) in enumerate(
                MultilabelStratifiedKFold(n_splits=n_kfold, random_state=seed, shuffle=True).split(target_df, target_df)
            ):
                print(f"Repeat: {seed}/{n_starts}, Fold: {n}/{n_kfold}")
    
                model = tf.keras.Sequential(
                    [tf.keras.layers.Input(len(top_feats))]
                    + layers
                    + [tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))]
                )

                model.compile(
                    optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                    loss="binary_crossentropy",
                )

                checkpoint_path = f"trial:{trial.number}_repeat:{seed}_fold:{n}.hdf5"
                reduce_lr_loss = ReduceLROnPlateau(
                    monitor="val_loss", factor=0.1, patience=3, verbose=0, epsilon=1e-4, mode="min"
                )
                cb_checkpt = ModelCheckpoint(
                    checkpoint_path, monitor="val_loss", verbose=0, save_best_only=True, save_weights_only=True, mode="min"
                )
                
                model.fit(
                    train.values[tr][:, top_feats],
                    target_df.values[tr],
                    validation_data=(train.values[te][:, top_feats], target_df.values[te]),
                    epochs=50,
                    batch_size=128,
                    callbacks=[reduce_lr_loss, cb_checkpt],
                    verbose=0,
                )

                model.load_weights(checkpoint_path)
                test_predict = model.predict(test.values[:, top_feats])
                val_predict = model.predict(train.values[te][:, top_feats])

                submit_df.loc[:, target_df.columns] += test_predict
                res.loc[te, target_df.columns] += val_predict

        submit_df.loc[:, target_df.columns] /= (n + 1) * n_starts
        res.loc[:, target_df.columns] /= n_starts
        
        self._submit_df = submit_df
        score = metric(target_df, res)
        
        return score
        
    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_submit_df = self._submit_df.copy()

In [20]:
objective = Objective()
study = optuna.create_study()
study.optimize(objective, n_trials=20, callbacks=[objective.callback])

n_starts: 2, n_kfold: 6, n_layers: 8


/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error



Repeat: 0/2, Fold: 0/6
Repeat: 0/2, Fold: 1/6
Repeat: 0/2, Fold: 2/6
Repeat: 0/2, Fold: 3/6
Repeat: 0/2, Fold: 4/6
Repeat: 0/2, Fold: 5/6


/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error



Repeat: 1/2, Fold: 0/6
Repeat: 1/2, Fold: 1/6
Repeat: 1/2, Fold: 2/6
Repeat: 1/2, Fold: 3/6
Repeat: 1/2, Fold: 4/6
Repeat: 1/2, Fold: 5/6


...

/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=8 as keyword args. From version 0.25 passing these as positional arguments will result in an error



Repeat: 8/10, Fold: 0/10
Repeat: 8/10, Fold: 1/10
Repeat: 8/10, Fold: 2/10
Repeat: 8/10, Fold: 3/10
Repeat: 8/10, Fold: 4/10
Repeat: 8/10, Fold: 5/10
Repeat: 8/10, Fold: 6/10
Repeat: 8/10, Fold: 7/10
Repeat: 8/10, Fold: 8/10
Repeat: 8/10, Fold: 9/10


/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=9 as keyword args. From version 0.25 passing these as positional arguments will result in an error



Repeat: 9/10, Fold: 0/10


KeyboardInterrupt: 

## 結果

In [ ]:
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
optuna.importance.get_param_importances(study)

In [ ]:
objective.best_submit_df.loc[test["cp_type"] == 1, target_df.columns] = 0

In [ ]:
objective.best_submit_df.to_csv("submission.csv", index=False)